In [1]:
import requests
import warnings
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
from langdetect import detect
from selenium import webdriver
import os
from googlesearch import search
import time


warnings.filterwarnings("ignore", message="Unverified HTTPS request")
driver = webdriver.Chrome()

# pip install requests
# pip install beautifulsoup4
# pip install pandas
# pip install langdetect
# pip install google


In [31]:
def removeHtmlTags(htmlText):
    soup = BeautifulSoup(htmlText, 'html.parser')
    text = soup.get_text(separator=" ", strip=True)
    words = text.split()
    # Filter out words containing "https://"
    filtered_words = [word for word in words if "https://" not in word and "http://" not in word]
    filtered_text = ' '.join(filtered_words)
    return filtered_text


def extractText(url):
    res = requests.get(url, verify=False)
    if res.status_code == 200:
        htmlPage = res.content
        soup = BeautifulSoup(htmlPage, 'html.parser')
        visibleText = soup.get_text()
        visibleText = removeHtmlTags(visibleText)
        lines = visibleText.split('\n')
        # i check for '.' and for a line longer that 200 chars to eliminate titles, headers
        text = '\n'.join(line.strip() for line in lines if len(line.strip()) > 0 and len(line) > 200 and '.' in line)
        return text
    return None

def canTextBeExtracted(url):
    res = requests.get(url, verify=False)
    if res.status_code == 200:
        htmlPage = res.content
        soup = BeautifulSoup(htmlPage, 'html.parser')
        visibleText = soup.get_text()
        lines = visibleText.split('\n')
        text = ' '.join(line.strip() for line in lines if len(line.strip()) > 0 and len(line) > 200 and '.' in line)
        text = removeHtmlTags(text)
        # i choose articles longer than 1000 chars
        if len(text) > 1000 and detect(text) == 'ro':
            return True
    return False

from urllib.parse import urlparse, urljoin

def findLinks(url, sitesToVisit, visitedSites):
    base_domain = urlparse(url).netloc.lower()
    blacklist = ['youtube.com', 'instagram.com', 'pinterest.com', 'twitter.com', 'facebook.com',
                 'login', 'cookie', 'cookies', 'politica-de-confidentialitate', 'despre-noi', 
                 'termeni-si-conditii', 'contact', 'privacy-policy', 'search', 'archive', 
                 'tag', 'category', 'forum', 'login', 'register', 'profile', 'logout', 
                 'sign-up','log-in','my-account','privacy','conditii','service','terms',
                 'conditions','about','sitemap','cont','comments','feed','politica-editoriala','cum-ne-poti-ajuta',
                 'password','paywall','arhiva','archive','termeni','despre','admin','newsletter',
                 'cart', 'checkout', 'shop', 'store', 'download', 'subscribe', 'unsubscribe','produs','abonare'
                 'terms-of-service', 'about-us', 'faq', 'donate', 'events', 'calendar', 
                 'faq', 'gallery', 'help', 'guidelines', 'policy']
    res = requests.get(url, verify=False)
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'html.parser')
        navbar = soup.find('nav', class_='navbar')
        footer = soup.find('footer', class_='footer')
        links = soup.find_all('a', href=True)
        for link in links:
            if navbar and link in navbar.find_all('a') or footer and link in footer.find_all('a'):
                continue
            href = link['href']
            if href.startswith('#') or href.startswith('javascript:'):
                continue  
            currentLink = urljoin(url, href)
            current_domain = urlparse(currentLink).netloc.lower()
            if current_domain == base_domain:
                if currentLink not in visitedSites and currentLink not in sitesToVisit and all(b not in currentLink.lower() for b in blacklist):
                    sitesToVisit.append(currentLink)
        return 0
    return 1

def extractAll(nrOfSitesToVisit, url, file_name):
    data = pd.DataFrame(columns=['link', 'Text'])
    sitesToVisit = []
    visitedSites = []
    initialSitesToVisit = nrOfSitesToVisit
    tries = 1
    findLinksCode = findLinks(url, sitesToVisit, visitedSites)
    startTime = time.time()
    if findLinksCode == 0:
        while nrOfSitesToVisit:
            if sitesToVisit:
                link = sitesToVisit[-1]
                extractableText = canTextBeExtracted(link)
                if extractableText:
                    text = extractText(link)
                    if text:
                        if not any(data['Text'].isin([text])):
                            new_row = {'link': link, 'Text': text}
                            data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)
                            nrOfSitesToVisit -= 1
                visitedSites.append(link)
                sitesToVisit.pop()
                print("Visited site: ",link)
            elif nrOfSitesToVisit:
                if len(visitedSites) >= tries and tries <= 2 * initialSitesToVisit:
                    findLinksCode = findLinks(visitedSites[-tries], sitesToVisit, visitedSites)
                    if findLinksCode == 1:
                        tries += 1
                else:
                    print('Only', initialSitesToVisit - nrOfSitesToVisit, 'articles out of', initialSitesToVisit, 'for', url)
                    nrOfSitesToVisit = 0
            endTime = time.time()
            if endTime - startTime > 180:
                print("3 minutes time limit exceeded")
                if not os.path.isfile(file_name):
                    data.to_csv(file_name, index=False)
                else:
                    data.to_csv(file_name, index=False, mode='a', header=False)
                return
        if not os.path.isfile(file_name):
            data.to_csv(file_name, index=False)
        else:
            data.to_csv(file_name, index=False, mode='a', header=False)
        return



def processUrls(urls, fileName):

    with open(fileName, 'w'):
        pass

    allData = pd.DataFrame(columns=['link', 'Text'])
    for url in urls:
        try:
            data = extractAll(10, url,fileName)
            allData = pd.concat([allData, data], ignore_index=True) 
        except Exception as e:
            print(f"Error occurred while processing {url}: {e}")
    return allData


In [4]:
def findBlogs(query, language, numResults=100):
    urls = []
    try:
        searchResults = search(query, num=numResults, lang=language, stop=numResults)
        for result in searchResults:
            urls.append(result)
    except Exception as e:
        print("An error occurred:", e)
    return urls

In [6]:
#incercare duckduckgo

def find_blogs(query, language, num_results=500):
    urls = []
    try:
        # DuckDuckGo Instant Answer API URL
        api_url = 'https://api.duckduckgo.com/'
        
        # Parameters for the DuckDuckGo search query
        params = {
            'q': query,
            'format': 'json',
            'no_redirect': 1,  # Disable redirection
            't': 'h_',  # Only search for HTML results (websites)
            'kl': language,  # Language parameter
            'skip_disambig': '1',  # Skip disambiguation (ambiguous) pages
            'no_html': '0',  # Include HTML in response
            'no_redirect': '1',  # Disable redirects
            'no_js': '0',  # Enable JavaScript (for dynamic content)
            'kp': '-2',  # Disable Safe Search
            'no_results': '0',  # Include search results
            's': '0',  # Start at the first result
            'norc': '1',  # Disable the DuckDuckGo search history feature
            'n': num_results  # Number of results to retrieve
        }
        
        # Send a GET request to the DuckDuckGo Instant Answer API
        response = requests.get(api_url+query, params=params)
        data = response.json()
        
        # Extract URLs from the response
        for result in data.get('Results', []):
            if 'FirstURL' in result:
                urls.append(result['FirstURL'])
    
    except Exception as e:
        print("An error occurred:", e)
    
    return urls



In [7]:
duckMd = find_blogs("site:.ro parenting blog","md")
print("500 Md Parenting Blogs:")
for i, blog in enumerate(duckMd, 1):
    print(f"{i}. {blog}")


500 Md Parenting Blogs:


In [23]:
query = "site:.ro parenting blog"
parentingBlogsRo = findBlogs(query, "ro")
print("100 Romanian Parenting Blogs:")
for i, blog in enumerate(parentingBlogsRo, 1):
    print(f"{i}. {blog}")


100 Romanian Parenting Blogs:
1. https://www.printesaurbana.ro/category/parenting
2. https://www.parentingconstient.ro/blog/
3. https://mamicaurbana.ro/
4. https://parentool.ro/blog/
5. https://zoso.ro/articole/parenting/
6. https://www.ceaicumamici.ro/
7. https://suntpitic.ro/
8. https://www.recenziidetop.ro/bloguri-parentale/
9. https://www.parentingineradigitala.ro/blog
10. https://www.bebelorelli.ro/blog/parenting
11. https://www.clubulcopiilor.ro/tema/parenting/
12. https://parentingpr.ro/blog/
13. https://lumealuifram.ro/blog/
14. https://mamalapatrat.ro/blog/
15. https://www.printesaurbana.ro/search/label/carti-de-parenting
16. https://www.parentingineradigitala.ro/blog?tag=greseli+de+parenting
17. https://clinica-hope.ro/blog-parenting/
18. https://uraniacremene.ro/blog/
19. https://parentool.ro/parenting/
20. http://blog.pandoram.ro/category/parenting/
21. https://www.parentingads.ro/
22. https://www.bebelorelli.ro/blog
23. https://www.blog.bloomcoding.ro/category/parenting/
2

In [26]:
parentingArticlesRo = processUrls(parentingBlogsRo,'parenting_articles_ro.csv')

Visited site:  https://www.parentingconstient.ro/blog/page/12/
Visited site:  https://www.parentingconstient.ro/blog/page/2/
Visited site:  https://www.parentingconstient.ro/2023/01/10/spiritualitatea-intre-concept-si-traire/
Visited site:  https://www.parentingconstient.ro/2023/01/25/este-in-inima-noastra/
Visited site:  https://www.parentingconstient.ro/author/claudia/
Visited site:  https://www.parentingconstient.ro/2023/03/28/transpersonal/
Visited site:  https://www.parentingconstient.ro/2023/06/21/omul-sfinteste-locul/
Visited site:  https://www.parentingconstient.ro/2023/07/11/moartea-subiect-tabu/
Visited site:  https://www.parentingconstient.ro/2023/08/15/cum-ne-crestem-copiii/
Visited site:  https://www.parentingconstient.ro/author/anghel/
Visited site:  https://www.parentingconstient.ro/2023/08/31/fortareata/
Visited site:  https://mamicaurbana.ro/gdpr-request-personal-data/%20
Visited site:  https://mamicaurbana.ro/am-testat-metoda-wim-hof-si-am-intrat-in-gheata/
Visited si

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Visited site:  http://parentingpr.ro/wp-content/uploads/2019/03/Parenting-PR-Presentation-2019.pdf
Visited site:  https://parentingpr.ro/2020/05/08/proiect-de-sustinere-a-business-urilor-romanesti/
Visited site:  https://parentingpr.ro/2020/06/26/interviu-cu-anton-balint-fondator-greenarium-ro/
Visited site:  https://parentingpr.ro/2020/07/23/sunday-bites-produce-in-romania-6-unturi-curate-din-ingrediente-naturale/
Visited site:  https://parentingpr.ro/2020/08/11/intr-o-lume-a-fast-fashionului-noi-am-ales-sa-fim-slow-oana-solomon-co-fondator-iya/
Visited site:  https://parentingpr.ro/2020/12/11/parentingpr-rgpd-cum-pot-gasi-un-specialist-in-domeniu/
Visited site:  https://parentingpr.ro/2021/02/03/pachete-gdpr-cum-sa-implementezi-prevederilor-din-domeniul-datelor-personale/
Visited site:  https://parentingpr.ro/2021/02/09/drepturile-gdpr-sau-cum-ne-protejam-clientii/
Visited site:  https://parentingpr.ro
Visited site:  https://parentingpr.ro/blog/
Visited site:  https://parentingpr.ro/

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Visited site:  https://uraniacremene.ro/wp-content/uploads/2020/12/Zizin-2.mp4
3 minutes time limit exceeded
Visited site:  https://parentool.ro/parenting/page/3/
Visited site:  https://parentool.ro/parenting/page/2/
Visited site:  https://parentool.ro/aplicatia-parentool-sprijin-parinti/
Visited site:  https://parentool.ro/obiceiuri-alimentare-sanatoase-pentru-copil/
Visited site:  https://parentool.ro/stresul-de-sarbatori/
Visited site:  https://parentool.ro/sarbatori-fara-tantrumuri/
Visited site:  https://parentool.ro/reuniunile-de-familie-de-sarbatori/
Visited site:  https://parentool.ro/scrisoare-tantrum/
Visited site:  https://parentool.ro/spiritul-de-sarbatori/
Visited site:  https://parentool.ro/pedofilia-cum-ne-protejam-copiii-online-offline/
Visited site:  http://blog.pandoram.ro/doar-baietii-sensibili-plang/
Visited site:  http://blog.pandoram.ro/povesti-terapeutice-pentru-copii/
Visited site:  http://blog.pandoram.ro/3-retete-delicioase-de-savurat-vara-asta/
Visited site: 

...

Visited site:  https://primusacademy.ro/imagine-stire/descopera-modelul-de-parenting-mastromarino-7.png
Visited site:  https://primusacademy.ro/blog
Visited site:  https://primusacademy.ro/
Visited site:  https://primusacademy.ro/blog/descopera-ti-potentialul-cu-analiza-tranzactionala-10-02-2024
Visited site:  https://primusacademy.ro/blog/descopera-modelul-de-parenting-mastromarino-25-03-2024
Visited site:  https://primusacademy.ro/blog/descopera-comunitatea-formativa-primus-01-04-2024
Visited site:  https://primusacademy.ro/linkedin.com/in/primus-agency-4471602b0
Visited site:  https://primusacademy.ro/blog/zi-de-formare-in-at-04-04-2024
Visited site:  https://primusacademy.ro/parenting-mastromarino
Visited site:  https://primusacademy.ro/curs-101-at/materiale
Visited site:  https://primusacademy.ro/curs-101-at
3 minutes time limit exceeded
Visited site:  http://stemcellsbank.ro/celulestem/?utm_source=Blog&utm_medium=link_ban_dr&utm_campaign=StemPV
Visited site:  http://stemcellsbank

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Visited site:  https://thewoman.ro/wp-content/uploads/2019/05/URMA-VOUCHER-50-LEI.jpg


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

3 minutes time limit exceeded
Visited site:  https://www.litera.ro/bookstops/
Visited site:  https://www.litera.ro/cariere/
Visited site:  https://www.litera.ro/blog/cuvocetare-i-ioana-baldea-constantinescu-in-dialog-cu-roxana-voloseniuc/
Visited site:  https://www.litera.ro/blog/citeste-un-fragment-in-avanpremiera-din-portret-de-familie-de-ann-napolitano/
Visited site:  https://www.litera.ro/blog/citeste-un-fragment-in-avanpremiera-din-vipere-la-patru-ace-de-paul-babiak-robert-d-hare/
Visited site:  https://www.litera.ro/blog/autori/bedros-horasangian/
Visited site:  https://www.litera.ro/blog/citeste-un-fragment-in-avanpremiera-din-eu-raman-aici-de-marco-balzano/
Visited site:  https://www.litera.ro/blog/citeste-un-fragment-in-avanpremiera-din-misiune-fara-sfarsit-de-david-goggins/
Visited site:  https://www.litera.ro/blog/citeste-un-fragment-in-avanpremiera-din-orasul-semilunii-casa-focului-si-a-umbrei/
Visited site:  https://www.litera.ro/blog/lansarea-romanului-castelul-din-orasul

In [5]:
query = "site:.md parenting blog"
parentingBlogsMd = findBlogs(query, "md")
print("100 Moldovan Parenting Blogs:")
for i, blog in enumerate(parentingBlogsMd, 1):
    print(f"{i}. {blog}")
#do not delete!!!!!!!!!!

100 Moldovan Parenting Blogs:
1. https://www.educatieparentala.md/
2. https://blog.blogtop.md/25-bloguri-de-parenting-din-moldova/
3. https://ea.md/tag/blog-de-parenting/
4. https://odoras.md/articole/topuri/topul-celor-mai-interesante-bloguri-de-parenting-din-romania/
5. https://smartkids.md/tag/parenting/
6. https://www.twinkl.md/blog/helping-my-child-to-form-their-numbers
7. https://blog.blogtop.md/category/uncategorized/
8. https://www.drepturilecopilului.md/index.php?option=com_content&view=category&layout=blog&id=85&Itemid=836&lang=en
9. https://iticket.md/event/bloggerita-printesa-urbana-despre-parenting-cu-iubire
10. https://diez.md/2019/12/07/ai-nevoie-de-sfaturi-sau-vrei-sa-cunosti-experienta-altor-parinti-iata-25-bloguri-de-parenting-din-moldova/
11. https://tibidoo.md/blog/page/2/
12. https://wb.md/2Gxpcf9
13. https://ea.md/baietelul-ei-a-inspirat-o-sa-creeze-un-blog-cunoaste-o-pe-a-blonde-mommy-mariana-mereuta-foto/
14. https://www.twinkl.md/blog/modern-art-a-guide-for-par

In [32]:
parentingArticlesMd = processUrls(parentingBlogsMd,'parenting_articles_md.csv')

Only 0 articles out of 10 for https://www.educatieparentala.md/
Visited site:  https://blog.blogtop.md/2017/04/
Visited site:  https://blog.blogtop.md/2017/07/
Visited site:  https://blog.blogtop.md/2017/09/
Visited site:  https://blog.blogtop.md/2017/10/
Visited site:  https://blog.blogtop.md/2017/11/
Visited site:  https://blog.blogtop.md/2017/12/
Visited site:  https://blog.blogtop.md/2018/01/
Visited site:  https://blog.blogtop.md/2018/02/
Visited site:  https://blog.blogtop.md/2018/03/
Visited site:  https://blog.blogtop.md/2018/04/
Visited site:  https://ea.md/cdn-cgi/l/email-protection
Visited site:  https://ea.md/prima-vacanta-cu-bebelusul-cateva-recomandari-utile-de-la-taticul-modern/
Visited site:  https://ea.md/publicitate/
Visited site:  https://ea.md/
Visited site:  https://ea.md/doliu-in-lumea-modei-roberto-cavalli-s-a-stins-din-viata-designerul-avea-83-de-ani/
Visited site:  https://ea.md/festivalul-animalelor-de-companie-de-manuta-cu-labuta-afla-ce-activitati-vor-avea-l

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Visited site:  https://eliteceramic.md/img/blog/unnamed.png


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Visited site:  https://eliteceramic.md/img/blog/pandasmall4442.jpg


...

Visited site:  https://eliteceramic.md/img/blog/17.970.png
Visited site:  https://eliteceramic.md/img/
Visited site:  https://eliteceramic.md/img/blog/?C=D;O=A
Visited site:  https://eliteceramic.md/img/blog/?C=S;O=A
Visited site:  https://eliteceramic.md/img/blog/?C=M;O=A
Visited site:  https://eliteceramic.md/img/blog/?C=N;O=D
Visited site:  https://eliteceramic.md/img/blog/?C=N;O=A
3 minutes time limit exceeded
Only 0 articles out of 10 for https://www.adservio.md/
Visited site:  https://imprimare.md/2015/09/17/another-top-london-blogs-award-for-love-london/#respond
Visited site:  https://imprimare.md/2016/03/28/how-does-playing-sports-make-you-more-healthy/
Visited site:  https://imprimare.md/2016/06/01/salut-lume/
Visited site:  https://imprimare.md/2016/09/22/diplome-pe-metal/
Visited site:  https://imprimare.md/2015/09/17/travels-a-long-weekend-in-mevagissey-cornwall-part-one/
Visited site:  https://imprimare.md/portfoliu/
Visited site:  https://imprimare.md/servicii-2/
Visited 

C:\Users\ilinc\AppData\Local\Temp\ipykernel_2492\1413987854.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(htmlText, 'html.parser')


Visited site:  https://imprimare.md/textile/camasi/
Visited site:  https://imprimare.md/tricouri/
Visited site:  https://imprimare.md/tricouri-copii/
3 minutes time limit exceeded
Visited site:  https://gi.md/locations/madison-ms
Visited site:  https://gi.md/locations/vicksburg-ms
Visited site:  https://gi.md/locations/flowood-ms
Visited site:  https://gi.md/Procedures
Visited site:  https://gi.md/resource/categories/allergies
Visited site:  https://gi.md/resource/categories/ibs
Visited site:  https://gi.md/resource/categories/gi-issues
Visited site:  https://gi.md/resource/categories/diarrhea
Visited site:  https://gi.md/resource/categories/celiac-disease
Visited site:  https://gi.md/resource/categories/gastric-cancer
Visited site:  https://gi.md/resource/categories/gerd
Visited site:  https://gi.md/resource/categories/ulcerative-colitis
Visited site:  https://gi.md/resource/categories/colon-cancer
Visited site:  https://gi.md/resource/categories/probiotics
Visited site:  https://gi.m

C:\Users\ilinc\AppData\Local\Temp\ipykernel_2492\1413987854.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(htmlText, 'html.parser')


Visited site:  http://blog.doni.md/2010/01/
Visited site:  http://blog.doni.md/2010/02/
Visited site:  http://blog.doni.md/2010/03/
Visited site:  http://blog.doni.md/2010/04/
Visited site:  http://blog.doni.md/2010/05/
Visited site:  http://blog.doni.md/2010/06/
Visited site:  http://blog.doni.md/2010/07/
Visited site:  http://blog.doni.md/2010/08/
Visited site:  http://blog.doni.md/2010/09/
Visited site:  http://blog.doni.md/2010/10/
Visited site:  http://blog.doni.md/2010/11/
Visited site:  http://blog.doni.md/2010/12/
Visited site:  http://blog.doni.md/2010/
Visited site:  http://blog.doni.md/2011/01/
Visited site:  http://blog.doni.md/2011/02/
Visited site:  http://blog.doni.md/2011/03/
Visited site:  http://blog.doni.md/2011/05/
Visited site:  http://blog.doni.md/2011/08/
Visited site:  http://blog.doni.md/2011/09/
Visited site:  http://blog.doni.md/2011/10/
Visited site:  http://blog.doni.md/2011/11/
Visited site:  http://blog.doni.md/2011/12/
Visited site:  http://blog.doni.md/

In [33]:
query = "site:.md sport blog"
sportBlogsMd = findBlogs(query, "md")
print("100 Moldovan Sport Blogs:")
for i, blog in enumerate(sportBlogsMd, 1):
    print(f"{i}. {blog}")


100 Moldovan Sport Blogs:
1. https://sandugrecu.blogspot.md/
2. https://www.records.md/ro/blog/categories/sports-1
3. https://sportspirit.md/ro/blog
4. https://blog.antrenor.md/
5. https://fitnessmama.md/
6. https://acvilasport.md/blog
7. https://enersport.md/blog/
8. https://fmf.md/blog
9. https://personaltrainer.md/blog/
10. https://marinabraniste.md/nu-ai-nicio-scuza-cand-e-vorba-de-sanatatea-ta-eu-am-ales-sa-fac-sport-tu/
11. https://sportspirit.md/ro/blog/brand_JORDAN_SPORT_SPIRIT_MOLODVA
12. https://www.siteweb.md/blog/item/57-ce-reprezinta-totusi-un-blog-cum-cream-un-blog
13. https://sportmarket.md/RO/blog
14. https://blogosfera.md/Sport-c-51-1-romana.html
15. https://blog.antrenor.md/posts/tu-cum-te-motivezi-cand-nu-ai-chef-de-sport
16. https://listblog.socio.md/tag/sport
17. https://suntparinte.md/fit-mami-blogul-care-incurajeaza-tinerele-mamici-sa-faca-sport-pentru-a-avea-o-silueta-de-invidiat/
18. https://dinamo.md/blog/page/14/
19. https://sportzone.md/?route=extension/dyna

In [34]:
sportArticlesMd = processUrls(sportBlogsMd,'sport_articles_md.csv')

Only 0 articles out of 10 for https://sandugrecu.blogspot.md/
Visited site:  https://www.records.md/ro/blog/categories/sports-1/page/6
Visited site:  https://www.records.md/ro/blog/categories/sports-1/page/5
Visited site:  https://www.records.md/ro/blog/categories/sports-1/page/4
Visited site:  https://www.records.md/ro/blog/categories/sports-1/page/3
Visited site:  https://www.records.md/ro/blog/categories/sports-1/page/2
Visited site:  https://www.records.md/ro/post/cel-mai-scurt-timp-pentru-realizarea-a-100-de-scufundari-mai-adanci-de-10-m
Visited site:  https://www.records.md/ro/post/cel-mai-rapid-inotator-la-distanta-de-100-m-bras
Visited site:  https://www.records.md/ro/post/the-highest-number-of-push-ups-with-claps-in-1-minute
Visited site:  https://www.records.md/ro/post/most-hammer-hits-on-a-tyre
Visited site:  https://www.records.md/ro/post/the-longest-hold-is-50-kg-load
Visited site:  https://www.records.md/ro/post/the-highest-number-of-jumping-ropes-in-1-minute
Visited site

C:\Users\ilinc\AppData\Local\Temp\ipykernel_2492\1413987854.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(htmlText, 'html.parser')


Visited site:  https://acvilasport.md/fitness/haltere-si-greutati/haltere-greutati-si-bare
Visited site:  https://acvilasport.md/fitness/haltere-si-greutati/gantere
Visited site:  https://acvilasport.md/fitness/haltere-si-greutati/scaun-pentru-haltere
Visited site:  https://acvilasport.md/fitness/haltere-si-greutati
Visited site:  https://acvilasport.md/fitness/aparate-fitnes/accesorii-pentru-aparate-fitnes
Visited site:  https://acvilasport.md/fitness/aparate-fitnes/scaun-pentru-exercitii
Visited site:  https://acvilasport.md/fitness/aparate-fitnes/banda-de-alergat
Visited site:  https://acvilasport.md/fitness/aparate-fitnes/aparat-multifunctional
Visited site:  https://acvilasport.md/fitness/aparate-fitnes
Visited site:  https://acvilasport.md/fitness/fitness/aparatori-si-protectii-suporturi-corporale-fitnes
Visited site:  https://acvilasport.md/fitness/fitness/greutati-gantere-haltere-si-bare-fitnes
Visited site:  https://acvilasport.md/fitness/fitness/step-fitnes
Visited site:  htt

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Visited site:  https://www.siteweb.md/media/k2/items/cache/0548677e6432786dd8df61eb3aaec139_XL.jpg
Visited site:  https://www.siteweb.md/component/mailto/?tmpl=component&template=yoo_infinite&link=bb91570fa56552b321adea59d7ddd0cbb1514a63
Visited site:  https://www.siteweb.md/blog/item/57-ce-reprezinta-totusi-un-blog-cum-cream-un-blog?tmpl=component&print=1
Visited site:  https://www.siteweb.md/en/
Visited site:  https://www.siteweb.md/
Visited site:  https://www.siteweb.md/blog
Visited site:  https://www.siteweb.md/seo-moldova
Visited site:  https://www.siteweb.md/design-web
Visited site:  https://www.siteweb.md/preturi
Visited site:  https://www.siteweb.md/portofoliu
Visited site:  https://www.siteweb.md
Visited site:  https://www.siteweb.md/\"\"


...

Visited site:  https://poka2ski.md/moto-velcro/
Visited site:  https://poka2ski.md/moto-cribs/
Visited site:  https://poka2ski.md/ro/2023/05/16/2023-05-14-deschiderea-arenei-tms-training-moto-skills/
Visited site:  https://poka2ski.md/ro/2023/06/06/2023-06-04-ziua-protectiei-copiilor-lacul-komsomolskoye/
Visited site:  https://poka2ski.md/ro/2023/06/08/moto-guzzi-audace-carbon-2016/
Visited site:  https://poka2ski.md/ro/2023/06/08/yamaha-xv1700pc-road-star-warrior/
Visited site:  https://poka2ski.md/ro/2023/06/08/honda-nc700x-dct-automat/
Visited site:  https://poka2ski.md/ro/2024/02/28/moto-guzzi-griso-8v-1200se/
Visited site:  https://poka2ski.md/ro/echipament-moto/
Visited site:  https://poka2ski.md/ro/moto-merchandise/
Visited site:  https://poka2ski.md/ro/moto-safety/
Visited site:  https://poka2ski.md/ro/evenimente/
Visited site:  https://poka2ski.md/ro/blog/
Visited site:  https://poka2ski.md/ro/
Visited site:  https://poka2ski.md/ro/2023/03/10/2023-03-08-8-martie-moto-flash-mob

C:\Users\ilinc\AppData\Local\Temp\ipykernel_2492\1413987854.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(htmlText, 'html.parser')


Visited site:  http://www.winninghealth.md/blog/2015/4/13/do-you-need-surgery#commenting
Visited site:  http://www.winninghealth.md/blog/2015/4/13/do-you-need-surgery
Visited site:  http://www.winninghealth.md/blog/2015/6/25/how-to-find-the-right-running-shoe#commenting
Visited site:  http://www.winninghealth.md/blog/2015/6/25/how-to-find-the-right-running-shoe
Visited site:  http://www.winninghealth.md/blog/2015/6/29/why-not-use-your-body-to-heal-itself#commenting
Visited site:  http://www.winninghealth.md/blog/2015/6/29/why-not-use-your-body-to-heal-itself
Visited site:  http://www.winninghealth.md/blog/2015/7/13/benefit-of-knee-surgery-for-middle-aged-or-older-patients-inconsequential-says-experts#commenting
Visited site:  http://www.winninghealth.md/blog/2015/7/13/benefit-of-knee-surgery-for-middle-aged-or-older-patients-inconsequential-says-experts
Visited site:  http://www.winninghealth.md/blog/2019/9/30/what-exactly-do-you-do-at-winning-health-sports-medicine#commenting
Visited 

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Only 0 articles out of 10 for https://www.orange.md/files/pdfs/other/noutati/lista-platforme-neautorizate-jocuri-de-noroc.pdf
Visited site:  https://myecobiz.md/client/etsy/
Visited site:  https://myecobiz.md/client/ebay/
Visited site:  https://myecobiz.md/client/amazon/
Visited site:  https://myecobiz.md/?page_id=473
Visited site:  https://myecobiz.md/cdn-cgi/l/email-protection
Visited site:  https://myecobiz.md/product/costum-de-baie-pentru-femei/
Visited site:  https://myecobiz.md/product/incaltamintea-sport-marathon/
Visited site:  https://myecobiz.md/product/cizme-de-calatorie/
Visited site:  https://myecobiz.md/product/pantaloni-scurti-negri-pentru-femei/
Visited site:  https://myecobiz.md/product/tricou-simplu-alb-pentru-barbati/
Visited site:  https://myecobiz.md/product/tank-top-ul-pentru-femei/
Visited site:  https://myecobiz.md/product/tricou-de-alergare-pentru-femei/
Visited site:  https://myecobiz.md/product/tricou-gri-pentru-barbati/
Visited site:  https://myecobiz.md/201

In [13]:
### am incercat cautari cu proxies ca sa nu mai fiu limitata de cate cautari pot face, nu a mers

# import time
# import requests
# from bs4 import BeautifulSoup

# def findBlogsWithProxies(query, language, proxy, numResults=1):
#     urls = []
#     try:
#         for _ in range(numResults):
#             searchResults = google_search(query, language, proxy, num=1)
#             for result in searchResults:
#                 urls.append(result)
#             time.sleep(1)
#     except Exception as e:
#         print("An error occurred:", e)
#     return urls

# def google_search(query, language, proxy, num=10):
#     url = f"https://www.google.com/search?q={query}&num={num}&hl={language}"
#     headers = {
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36"
#     }

#     response = requests.get(url, headers=headers, proxies={'http': proxy, 'https': proxy})
    
#     if response.status_code == 200:
#         urls = []
#         soup = BeautifulSoup(response.text, 'html.parser')
#         for result in soup.find_all('div', class_='tF2Cxc'):
#             title = result.find('h3').text
#             link = result.find('a')['href']
#             urls.append({'title': title, 'link': link})
#         return urls
#     else:
#         return []



In [14]:
### am incercat cautari cu proxies ca sa nu mai fiu limitata de cate cautari pot face, nu a mers

# query = "site:.ro parenting blog"
# proxy = "http://43.255.113.232:83"
# parentingBlogsRo = findBlogsWithProxies(query, "ro", proxy)
# print("100 Romanian Parenting Blogs:")
# for i, blog in enumerate(parentingBlogsRo, 1):
#     print(f"{i}. {blog}")


An error occurred: HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: /search?q=site:.ro%20parenting%20blog&num=1&hl=ro (Caused by ProxyError('Unable to connect to proxy', ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002193BE4EE50>, 'Connection to 43.255.113.232 timed out. (connect timeout=None)')))
100 Romanian Parenting Blogs:
